In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
try:
    df = pd.read_csv('fifa21 raw data v2.csv')
except FileNotFoundError:
    print('File Not Found! Please check filepath and try again!')
    raise

C:\Users\Maxwe\AppData\Local\Temp\ipykernel_23216\2568790447.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('fifa21 raw data v2.csv')


In [3]:
df.head(10)

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595
5,188545,R. Lewandowski,Robert Lewandowski,https://cdn.sofifa.com/players/188/545/21_60.png,http://sofifa.com/player/188545/robert-lewando...,Poland,31,91,91,\n\n\n\nFC Bayern München,...,High,Medium,4 ★,78,91,78,85,43,82,248
6,209331,M. Salah,Mohamed Salah,https://cdn.sofifa.com/players/209/331/21_60.png,http://sofifa.com/player/209331/mohamed-salah/...,Egypt,28,90,90,\n\n\n\nLiverpool,...,High,Medium,3 ★,93,86,81,90,45,75,246
7,212831,Alisson,Alisson Ramses Becker,https://cdn.sofifa.com/players/212/831/21_60.png,http://sofifa.com/player/212831/alisson-ramses...,Brazil,27,90,91,\n\n\n\nLiverpool,...,Medium,Medium,3 ★,86,88,85,89,51,91,120
8,231747,K. Mbappé,Kylian Mbappé,https://cdn.sofifa.com/players/231/747/21_60.png,http://sofifa.com/player/231747/kylian-mbappe/...,France,21,90,95,\n\n\n\nParis Saint-Germain,...,High,Low,3 ★,96,86,78,91,39,76,1.6K
9,192448,M. ter Stegen,Marc-André ter Stegen,https://cdn.sofifa.com/players/192/448/21_60.png,http://sofifa.com/player/192448/marc-andre-ter...,Germany,28,90,93,\n\n\n\nFC Barcelona,...,Medium,Medium,3 ★,88,85,88,90,45,88,130


# $Exploratory$ $Data$ $Analysis$

In [4]:
print('Number of observations : ',df.shape[0])
print('Number of features : ',df.shape[1])

Number of observations :  18979
Number of features :  77


In [5]:
print('--Data Info--')
df.info()

--Data Info--
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17

In [6]:
print('----------Numerical Summary-----------')
display(df.describe(exclude='number').T)

----------Numerical Summary-----------


,count,unique,top,freq
Name,18979,17920,J. Rodríguez,13
LongName,18979,18852,Danny Rose,3
photoUrl,18979,18979,https://cdn.sofifa.com/players/158/023/21_60.png,1
playerUrl,18979,18979,http://sofifa.com/player/158023/lionel-messi/2...,1
Nationality,18979,164,England,1705
Club,18979,682,No Club,237
Contract,18979,131,2019 ~ 2021,1706
Positions,18979,640,CB,2441
Height,18979,62,180cm,1474
Weight,18979,79,70kg,1495


In [7]:
print('------------Categorical Summary------------')
display(df.describe(include='number').T)

------------Categorical Summary------------


,count,mean,std,min,25%,50%,75%,max
ID,18979.0,226403.384794,27141.054157,41.0,210135.0,232418.0,246922.5,259216.0
Age,18979.0,25.194109,4.710520,16.0,21.0,25.0,29.0,53.0
↓OVA,18979.0,65.718636,6.968999,47.0,61.0,66.0,70.0,93.0
POT,18979.0,71.136414,6.114635,47.0,67.0,71.0,75.0,95.0
BOV,18979.0,66.751726,6.747193,48.0,62.0,67.0,71.0,93.0
Attacking,18979.0,248.938142,74.299428,42.0,222.0,263.0,297.0,437.0
Crossing,18979.0,49.688392,18.131153,6.0,38.0,54.0,63.0,94.0
Finishing,18979.0,45.842405,19.567081,3.0,30.0,49.0,62.0,95.0
Heading Accuracy,18979.0,51.942726,17.294409,5.0,44.0,55.0,64.0,93.0
Short Passing,18979.0,58.768112,14.519106,7.0,54.0,62.0,68.0,94.0


`Duplications`

In [8]:
print('-------Duplicates-----')
duplicates = df[df.duplicated()]
print('Number of duplicated rows : ',len(duplicates))
if len(duplicates) == 0:
    print('No duplicates found')
else:
    display(duplicates)

-------Duplicates-----
Number of duplicated rows :  0
No duplicates found


$Insight$ ~ `No duplicated observations`

`Missing Data`

In [9]:
def missing_data(df):
    missing_values = df.isnull().sum()
    missing_pct = (missing_values / len(df)) * 100
    missing_data = pd.DataFrame({
        'Missing Values' : missing_values,
        'Missing Pct' : missing_pct.round(2)
    }).sort_values(by='Missing Pct',ascending=False)
    return missing_data
missing = missing_data(df)
print(missing)

               Missing Values  Missing Pct
Loan Date End           17966        94.66
Hits                     2595        13.67
LongName                    0         0.00
Name                        0         0.00
ID                          0         0.00
...                       ...          ...
PAC                         0         0.00
PAS                         0         0.00
DRI                         0         0.00
DEF                         0         0.00
PHY                         0         0.00

[77 rows x 2 columns]


`Numeric column Analysis`

In [10]:
print('----------Numeric Columns--------\n')
numeric_cols = df.select_dtypes(include=[np.number]).columns
for i,col in enumerate(numeric_cols,1):
    print(f'{i:<2}. {col:<17} -Min : {df[col].min():<4} -Max : {df[col].max()}')

----------Numeric Columns--------

1 . ID                -Min : 41   -Max : 259216
2 . Age               -Min : 16   -Max : 53
3 . ↓OVA              -Min : 47   -Max : 93
4 . POT               -Min : 47   -Max : 95
5 . BOV               -Min : 48   -Max : 93
6 . Attacking         -Min : 42   -Max : 437
7 . Crossing          -Min : 6    -Max : 94
8 . Finishing         -Min : 3    -Max : 95
9 . Heading Accuracy  -Min : 5    -Max : 93
10. Short Passing     -Min : 7    -Max : 94
11. Volleys           -Min : 3    -Max : 90
12. Skill             -Min : 40   -Max : 470
13. Dribbling         -Min : 5    -Max : 96
14. Curve             -Min : 4    -Max : 94
15. FK Accuracy       -Min : 5    -Max : 94
16. Long Passing      -Min : 5    -Max : 93
17. Ball Control      -Min : 5    -Max : 96
18. Movement          -Min : 122  -Max : 464
19. Acceleration      -Min : 13   -Max : 97
20. Sprint Speed      -Min : 12   -Max : 96
21. Agility           -Min : 14   -Max : 96
22. Reactions         -Min : 24   

`Categorical Columns`

In [11]:
print('-------------Categorical Columns--------------\n')
categorical_cols = df.select_dtypes(exclude=[np.number]).columns
for i,col in enumerate(categorical_cols,1):
    print(f'{i}. {col} -{df[col].unique()} - {df[col].nunique()}')

-------------Categorical Columns--------------

1. Name -['L. Messi' 'Cristiano Ronaldo' 'J. Oblak' ... 'R. McKinley'
 "Wang Zhen'ao" 'Zhou Xiao'] - 17920
2. LongName -['Lionel Messi' 'C. Ronaldo dos Santos Aveiro' 'Jan Oblak' ...
 'Ronan McKinley' "Zhen'ao Wang" 'Xiao Zhou'] - 18852
3. photoUrl -['https://cdn.sofifa.com/players/158/023/21_60.png'
 'https://cdn.sofifa.com/players/020/801/21_60.png'
 'https://cdn.sofifa.com/players/200/389/21_60.png' ...
 'https://cdn.sofifa.com/players/252/757/21_60.png'
 'https://cdn.sofifa.com/players/243/790/21_60.png'
 'https://cdn.sofifa.com/players/252/520/21_60.png'] - 18979
4. playerUrl -['http://sofifa.com/player/158023/lionel-messi/210006/'
 'http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/210006/'
 'http://sofifa.com/player/200389/jan-oblak/210006/' ...
 'http://sofifa.com/player/252757/ronan-mckinley/210006/'
 'http://sofifa.com/player/243790/zhenao-wang/210006/'
 'http://sofifa.com/player/252520/xiao-zhou/210006/'] - 18979
5. Na

`Outlier Detection Using IQR`

In [12]:
def check_outliers(df,col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    return outlier, upper_bound, lower_bound
for col in numeric_cols:
    outlier,upper_bound, lower_bound = check_outliers(df,col)
    print(f'{i:<2}. {col:<18} - {len(outlier)} outliers (Range : {lower_bound:.2f} - {upper_bound:.2f})')

23. ID                 - 250 outliers (Range : 154953.75 - 302103.75)
23. Age                - 8 outliers (Range : 9.00 - 41.00)
23. ↓OVA               - 156 outliers (Range : 47.50 - 83.50)
23. POT                - 153 outliers (Range : 55.00 - 87.00)
23. BOV                - 137 outliers (Range : 48.50 - 84.50)
23. Attacking          - 2016 outliers (Range : 109.50 - 409.50)
23. Crossing           - 0 outliers (Range : 0.50 - 100.50)
23. Finishing          - 0 outliers (Range : -18.00 - 110.00)
23. Heading Accuracy   - 1108 outliers (Range : 14.00 - 94.00)
23. Short Passing      - 1748 outliers (Range : 33.00 - 89.00)
23. Volleys            - 0 outliers (Range : -9.00 - 95.00)
23. Skill              - 1233 outliers (Range : 90.00 - 442.00)
23. Dribbling          - 1962 outliers (Range : 20.50 - 96.50)
23. Curve              - 0 outliers (Range : -4.00 - 100.00)
23. FK Accuracy        - 3 outliers (Range : -5.00 - 91.00)
23. Long Passing       - 20 outliers (Range : 11.50 - 95.50)
23.